In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.model_selection as skms
import sklearn.metrics as skm
import sklearn.tree as sktree
import sklearn.ensemble as skensemble
import sklearn.datasets as skds
import sklearn.feature_selection as skfs
import sklearn.preprocessing as skpp

# Limpieza de datos

In [3]:
!wget lalussoft.com/mldemos/personas40s.csv

--2021-06-08 18:17:07--  http://lalussoft.com/mldemos/personas40s.csv
Resolving lalussoft.com (lalussoft.com)... 34.198.109.148
Connecting to lalussoft.com (lalussoft.com)|34.198.109.148|:80... connected.
HTTP request sent, awaiting response... 302 Redirect
Location: https://lalussoft.com/mldemos/personas40s.csv [following]
--2021-06-08 18:17:07--  https://lalussoft.com/mldemos/personas40s.csv
Connecting to lalussoft.com (lalussoft.com)|34.198.109.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18863 (18K) [application/octet-stream]
Saving to: ‘personas40s.csv’

personas40s.csv     100%[===================>]  18.42K  --.-KB/s    in 0s      

2021-06-08 18:17:07 (201 MB/s) - ‘personas40s.csv’ saved [18863/18863]



In [5]:
data = pd.read_csv("personas40s.csv")
data.pop("Unnamed: 0")
data

,es_hombre,altura,peso
0,0.0,154.0,58.0
1,1.0,180.0,84.0
2,1.0,155.0,67.0
3,1.0,181.0,89.0
4,1.0,164.0,65.0
...,...,...,...
995,1.0,178.0,80.0
996,0.0,175.0,91.0
997,1.0,177.0,80.0
998,1.0,172.0,77.0


In [6]:
data.isnull().sum()
#hay data que es nula

es_hombre    7
altura       2
peso         5
dtype: int64

In [8]:
#muestro cuales son nulos
data[ data["es_hombre"].isnull() ]

,es_hombre,altura,peso
70,NaN,157.0,51.0
174,NaN,146.0,55.0
336,NaN,161.0,66.0
499,NaN,166.0,70.0
621,NaN,175.0,80.0
912,NaN,175.0,85.0
950,NaN,164.0,72.0


In [9]:
#los quito
data = data.drop([70, 174, 336, 499, 621, 912, 950])

In [10]:
data[ data["es_hombre"].isnull() ]

,es_hombre,altura,peso


In [11]:
import sklearn.impute as skim

In [13]:
reemplazo = skim.SimpleImputer(missing_values=np.nan, strategy="mean")
#los datos faltantes los reemplazamos por las medias
data["alturac"] = reemplazo.fit_transform(data[["altura"]])
data["pesoc"] = reemplazo.fit_transform(data[["peso"]])

In [14]:
data

,es_hombre,altura,peso,alturac,pesoc
0,0.0,154.0,58.0,154.0,58.0
1,1.0,180.0,84.0,180.0,84.0
2,1.0,155.0,67.0,155.0,67.0
3,1.0,181.0,89.0,181.0,89.0
4,1.0,164.0,65.0,164.0,65.0
...,...,...,...,...,...
995,1.0,178.0,80.0,178.0,80.0
996,0.0,175.0,91.0,175.0,91.0
997,1.0,177.0,80.0,177.0,80.0
998,1.0,172.0,77.0,172.0,77.0


In [ ]:
data.pop("altura")
data.pop("peso")

In [16]:
data.isnull().sum()

es_hombre    0
alturac      0
pesoc        0
dtype: int64

In [17]:
data[data["alturac"]<50]

,es_hombre,alturac,pesoc
489,0.0,-28.0,65.0


In [19]:
data = data.drop(489)

In [20]:
data[data["alturac"]<50]

,es_hombre,alturac,pesoc


In [21]:
data[data["pesoc"]<30]

,es_hombre,alturac,pesoc
96,0.0,162.0,-21.0


In [22]:
data = data.drop(96)

In [23]:
data[data["pesoc"]<30]

,es_hombre,alturac,pesoc


In [26]:
xtrain, xtest, ytrain, ytest = skms.train_test_split(data[["alturac", "pesoc"]],
                                                     data["es_hombre"],
                                                     test_size=0.3, random_state=0)

Escalamiento MinMax

In [27]:
minmax = skpp.MinMaxScaler()

In [29]:
xtrain_escalado = minmax.fit_transform(xtrain)
xtrain_escalado

array([[0.69090909, 0.6984127 ],
       [0.38181818, 0.57142857],
       [0.74545455, 0.80952381],
       ...,
       [0.45454545, 0.55555556],
       [0.52727273, 0.53968254],
       [0.45454545, 0.50793651]])

In [30]:
xtest_escalado= minmax.fit_transform(xtest)

In [31]:
import sklearn.linear_model as sklm

In [32]:
modelo = sklm.LogisticRegression()
modelo.fit(xtrain_escalado, ytrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [33]:
modelo.score(xtrain_escalado, ytrain)

0.7518037518037518

In [34]:
modelo.score(xtest_escalado, ytest)

0.7416107382550335

In [35]:
skm.accuracy_score(ytest, modelo.predict(xtest_escalado))

0.7416107382550335

In [36]:
rfc = skensemble.RandomForestClassifier()
rfc.fit(xtrain_escalado, ytrain)
#skm.accuracy_score()

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

# Seleccion de Features

In [39]:
wine = skds.load_wine()
print(wine.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

In [42]:
data = pd.DataFrame(wine.data, columns=wine.feature_names)
data["target"] = pd.Series(wine.target)
data

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


In [43]:
xtrain,xtest,ytrain,ytest = skms.train_test_split(data[wine.feature_names],data["target"],
                                                  test_size=0.3,random_state=0)

Analisis de la varianza del feature

In [44]:
vt = skfs.VarianceThreshold(threshold=0)
xtrainvt = pd.DataFrame(vt.fit_transform(xtrain),columns=xtrain.columns)
xtrainvt.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,13.71,1.86,2.36,16.6,101.0,2.61,2.88,0.27,1.69,3.80,1.11,4.00,1035.0
1,12.22,1.29,1.94,19.0,92.0,2.36,2.04,0.39,2.08,2.70,0.86,3.02,312.0
2,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
3,13.16,3.57,2.15,21.0,102.0,1.50,0.55,0.43,1.30,4.00,0.60,1.68,830.0
4,13.86,1.51,2.67,25.0,86.0,2.95,2.86,0.21,1.87,3.38,1.36,3.16,410.0


El siguiente análisis de varianza está afectado por la escala de los datos

In [45]:
df=pd.DataFrame(np.column_stack((xtrain.columns,vt.variances_)),columns=("features","varianza"))
df.sort_values("varianza",ascending=False)

,features,varianza
12,proline,1235
4,magnesium,92
3,alcalinity_of_ash,12.71
9,color_intensity,5.48412
1,malic_acid,1.28166
6,flavanoids,0.946921
0,alcohol,0.636966
11,od280/od315_of_diluted_wines,0.517446
5,total_phenols,0.357333
8,proanthocyanins,0.355007


Escalamos los datos para poder hacer una comparación adecuada

In [46]:
minmax = skpp.MinMaxScaler()
xtrain_escalado = pd.DataFrame(minmax.fit_transform(xtrain),columns=xtrain.columns)

In [47]:
xtrain_escalado.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,0.720430,0.203782,0.537634,0.309278,0.336957,0.543165,0.737003,0.250000,0.401899,0.240688,0.487179,1.000000,0.585425
1,0.319892,0.084034,0.311828,0.432990,0.239130,0.453237,0.480122,0.480769,0.525316,0.135626,0.273504,0.641026,0.000000
2,0.602151,0.712185,0.483871,0.484536,0.543478,0.176259,0.067278,0.557692,0.294304,0.851958,0.042735,0.106227,0.423482
3,0.572581,0.563025,0.424731,0.536082,0.347826,0.143885,0.024465,0.557692,0.278481,0.259790,0.051282,0.150183,0.419433
4,0.760753,0.130252,0.704301,0.742268,0.173913,0.665468,0.730887,0.134615,0.458861,0.200573,0.700855,0.692308,0.079352


In [48]:
vt = skfs.VarianceThreshold(threshold=0)
xtrainvt_escalado = pd.DataFrame(vt.fit_transform(xtrain_escalado),columns=xtrain.columns)
xtrainvt_escalado.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,0.720430,0.203782,0.537634,0.309278,0.336957,0.543165,0.737003,0.250000,0.401899,0.240688,0.487179,1.000000,0.585425
1,0.319892,0.084034,0.311828,0.432990,0.239130,0.453237,0.480122,0.480769,0.525316,0.135626,0.273504,0.641026,0.000000
2,0.602151,0.712185,0.483871,0.484536,0.543478,0.176259,0.067278,0.557692,0.294304,0.851958,0.042735,0.106227,0.423482
3,0.572581,0.563025,0.424731,0.536082,0.347826,0.143885,0.024465,0.557692,0.278481,0.259790,0.051282,0.150183,0.419433
4,0.760753,0.130252,0.704301,0.742268,0.173913,0.665468,0.730887,0.134615,0.458861,0.200573,0.700855,0.692308,0.079352


In [49]:
df=pd.DataFrame(np.column_stack((xtrain.columns,vt.variances_)),
                columns=("features","varianza"))
df.sort_values("varianza",ascending=False)

,features,varianza
6,flavanoids,0.088556
11,od280/od315_of_diluted_wines,0.0694288
12,proline,0.0620394
7,nonflavanoid_phenols,0.0566197
1,malic_acid,0.0565663
9,color_intensity,0.0500281
5,total_phenols,0.0462363
0,alcohol,0.0460289
10,hue,0.0402041
8,proanthocyanins,0.0355519


También podemos usar el F-Test

In [50]:
f,p = skfs.f_classif(xtrain_escalado,ytrain)

In [51]:
pd.DataFrame(np.column_stack((xtrain.columns,f,p)),
             columns=("features","F","p")).sort_values("F",ascending=False)

,features,F,p
6,flavanoids,217.586,8.38203e-41
11,od280/od315_of_diluted_wines,150.623,1.45154e-33
12,proline,143.569,1.13422e-32
9,color_intensity,87.7927,2.77621e-24
0,alcohol,85.3086,7.71473e-24
5,total_phenols,76.0424,4.09744e-22
10,hue,70.5972,4.80611e-21
3,alcalinity_of_ash,28.5458,6.99182e-11
1,malic_acid,27.766,1.19038e-10
7,nonflavanoid_phenols,24.0514,1.60467e-09


Finalmente, los modelos mismos nos dán información respecto a la importancia de las variables.

In [52]:
rfc = skensemble.RandomForestClassifier()
rfc.fit(xtrain_escalado,ytrain)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [53]:
pd.DataFrame(np.column_stack((xtrain.columns,f,p,rfc.feature_importances_)),
             columns=("features","F","p","Feat.Imp.RF")).sort_values(
                 "Feat.Imp.RF",ascending=False)

,features,F,p,Feat.Imp.RF
9,color_intensity,87.7927,2.77621e-24,0.185108
12,proline,143.569,1.13422e-32,0.184672
6,flavanoids,217.586,8.38203e-41,0.145925
11,od280/od315_of_diluted_wines,150.623,1.45154e-33,0.143189
0,alcohol,85.3086,7.71473e-24,0.0992385
10,hue,70.5972,4.80611e-21,0.0675819
5,total_phenols,76.0424,4.09744e-22,0.0624443
3,alcalinity_of_ash,28.5458,6.99182e-11,0.0284383
1,malic_acid,27.766,1.19038e-10,0.0216548
4,magnesium,3.36877,0.0376913,0.0210521


In [54]:
skm.accuracy_score(ytest,rfc.predict(minmax.transform(xtest)))

0.9814814814814815

In [55]:
skm.accuracy_score(ytrain,rfc.predict(xtrain_escalado))

1.0

Probamos los resultados eliminando el feature "ash" y "proline"

In [ ]:
xtrain.pop("ash")
xtest.pop("ash")

In [57]:
minmax = skpp.MinMaxScaler()
xtrain_escalado = pd.DataFrame(minmax.fit_transform(xtrain))
rfc = skensemble.RandomForestClassifier()
rfc.fit(xtrain_escalado,ytrain)
skm.accuracy_score(ytest,rfc.predict(minmax.transform(xtest)))

0.9629629629629629

In [58]:
xtrain.pop("proline")
xtest.pop("proline")
minmax = skpp.MinMaxScaler()
xtrain_escalado = pd.DataFrame(minmax.fit_transform(xtrain))
rfc = skensemble.RandomForestClassifier()
rfc.fit(xtrain_escalado,ytrain)
skm.accuracy_score(ytest,rfc.predict(minmax.transform(xtest)))

0.9444444444444444